## Собираем трансформер

In [1]:
%%capture
!pip install -q torchdata==0.3.0 torchtext==0.12 spacy==3.2 altair GPUtil
!python -m spacy download de_core_news_sm
!python -m spacy download en_core_web_sm

In [2]:
import os
import time
import math
import copy
import spacy
import GPUtil
import pandas as pd
from typing import *
from itertools import chain

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.distributed as dist
import torch.multiprocessing as mp
from torch.optim.lr_scheduler import LambdaLR
from torch.nn.parallel import DistributedDataParallel as DDP
from torch.utils.data import DataLoader, Dataset

import altair as alt
from altair import Chart

alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

## Positional Encoding

До появления архитектуры Transformer, модели для обработки последовательностей(такие как RNN и LSTM), обрабатывали данные последовательно, автоматически учитывая порядок элементов. Однако их вычислительная неэффективность из-за пошаговой обработки и сложности параллелизации привела к поиску альтернатив. 

Transformer, появившийся в 2017 году, устранил эти ограничения за счёт полностью параллельного подхода, но возникла новая проблема: модель не могла учитывать порядок элементов в последовательности, так как все токены обрабатывались одновременно. 

Для решения этой проблемы был введен **Positional Encoding** — механизм, кодирующий информацию о позиции каждого элемента.

Positional Encoding добавляет к векторным представлениям токенов (эмбеддингам) специальные сигналы, зависящие от их позиции в последовательности. Это позволяет модели различать слова "кошка" в позиции 1 и "кошка" в позиции 5, даже если их семантические эмбеддинги идентичны. Формула кодирования использует комбинацию синусов и косинусов с разными частотами:  
$$  
PE_{(pos, 2i)} = \sin\left(\frac{pos}{10000^{2i/d_{\text{model}}}}\right), \quad  
PE_{(pos, 2i+1)} = \cos\left(\frac{pos}{10000^{2i/d_{\text{model}}}}\right),  
$$  
где $pos$ — позиция элемента, $d_{\text{model}}$ — размерность эмбеддингов, $i$ — индекс измерения в векторе.  

Основная идея в том, что синусоидальные функции позволяют модели обращать внимание на **относительные позиции**. 

### Почему эта странная формула кодирует относительные позиции токенов?

Представьте, что каждая позиция в последовательности — это точка на числовой прямой. Если для позиции $pos$ мы генерируем сигналы с помощью синуса и косинуса, то для позиции $pos + k$ эти сигналы можно выразить через комбинацию исходных значений. Например, по формуле сложения углов:  
$$  
\sin(pos + k) = \sin(pos)\cos(k) + \cos(pos)\sin(k),  
$$  

Это означает, что смещение на $k$ позиций выражается через взвешенную сумму исходных синуса и косинуса. Это позволяет модели автоматически улавливать, что «слово через три позиции» связано с исходным словом, даже если она никогда не видела такую длинную последовательность при обучении.  

Логарифмическое убывание частот в формуле $10000^{2i/d_{\text{model}}}$ обеспечивает, что разные измерения вектора позиционного кодирования отвечают за разные уровни детализации позиции. При малых значениях $i$ (например, первые компоненты вектора) знаменатель $10000^{2i/d_{\text{model}}}$ становится большим, что замедляет рост аргумента синуса и косинуса при увеличении $pos$. Это создаёт низкочастотные колебания, которые позволяют различать позиции на больших масштабах: например, начало текста (позиции 1-100) от середины (позиции 101-200). При больших $i$ знаменатель уменьшается, аргумент функций растёт быстрее, и возникают высокочастотные колебания, которые кодируют тонкие различия между соседними позициями (например, 101 и 102).  

Чередование синусов и косинусов для чётных и нечётных индексов решает проблему уникальности позиционных кодировок. Если бы использовалась только синусоида, разные позиции могли бы случайно совпадать из-за периодичности функции (например, $\sin(pos)$ и $\sin(pos + 2\pi)$). Добавление косинуса для соседних компонент вектора устраняет эту симметрию: комбинация $\sin(f(pos))$ и $\cos(f(pos))$ для разных частот $f$ гарантирует, что каждая позиция $pos$ будет иметь уникальный вектор. Ортогональность синусов и косинусов (их скалярное произведение близко к нулю) минимизирует перекрытие с эмбеддингами слов, что позволяет модели раздельно обрабатывать семантику и позицию.  

Суммирование $\text{Embedding} + PE$ возможно, потому что эмбеддинги слов и позиционные кодировки имеют одинаковую размерность $d_{\text{model}}$. Это сложение не требует обучаемых параметров: модель получает объединённый сигнал, где семантика слова модифицируется в соответствии с его позицией. Градиенты распространяются через эту операцию без искажений, так как производная суммы равна сумме производных. В результате, во время обучения модель автоматически учится корректировать и семантические эмбеддинги, и использование позиционной информации (через механизм внимания), не сталкиваясь с конфликтом сигналов.  

Исследовались так же и альтернативные подходы, такие как обучаемые позиционные эмбеддинги. Но синусоидальная схема оказалась предпочтительнее из-за способности обобщаться на последовательности длиннее тех, что встречались при обучении. Таким образом, Positional Encoding стал компромиссом между выразительностью, вычислительной эффективностью и отсутствием дополнительных обучаемых параметров, что идеально соответствовало задаче параллельной обработки в Transformer.  

In [3]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout, max_len=5000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)
        
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() *
                             (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:, :x.size(1)].detach()
        return self.dropout(x)

## MultiHeadAttention

**MultiHeadAttention** в архитектуре Transformer возник как ответ на ограничения механизмов внимания, которые до этого использовались в моделях seq2seq. Изначально самовнимание (self-attention) позволяло каждому элементу последовательности взаимодействовать с другими, вычисляя взвешенные суммы их признаков. Однако проблема заключалась в том, что одно "головное" внимание (single head) могло фокусироваться только на одном типе зависимостей — например, на синтаксических связях или семантической близости. Для сложных задач, таких как перевод, требовалось одновременно учитывать **разнородные взаимодействия**: связи между подлежащим и сказуемым, анафоры, контекстные синонимы и т.д.  

**Решение**: вместо одного механизма внимания использовать несколько параллельных "голов" (heads), каждая из которых учится выделять свой тип зависимостей. Формально, для входных векторов (эмбеддингов) $X \in \mathbb{R}^{n \times d_{\text{model}}}$, где $n$ — длина последовательности, а $d_{\text{model}}$ — размерность эмбеддингов, каждая голова $h$ проецирует $X$ в три пространства — запросов ($Q_h$), ключей ($K_h$) и значений ($V_h$) — через обучаемые матрицы весов:  
$$  
Q_h = X W_h^Q, \quad K_h = X W_h^K, \quad V_h = X W_h^V,  
$$  
где $W_h^Q, W_h^K \in \mathbb{R}^{d_{\text{model}} \times d_k}$, $W_h^V \in \mathbb{R}^{d_{\text{model}} \times d_v}$, а $d_k$ и $d_v$ — размерности подпространств для ключей/запросов и значений. Выбор трёх матриц ($Q, K, V$) обусловлен аналогией с информационным поиском:  
- **Запросы** ($Q$) — что ищем,  
- **Ключи** ($K$) — по чему ищем,  
- **Значения** ($V$) — что возвращаем.  

Если бы использовались только $Q$ и $K$, модель не смогла бы преобразовать найденные зависимости в новые признаки. Матрица $V$ добавляет гибкость, позволяя перевзвешивать значения в соответствии с контекстом.  

Для каждой головы вычисляется **масштабированное скалярное произведение** (scaled dot-product attention):  
$$  
\text{Attention}(Q_h, K_h, V_h) = \text{softmax}\left(\frac{Q_h K_h^T}{\sqrt{d_k}}\right) V_h.  
$$  
**Почему softmax?** Функция softmax преобразует неограниченные оценки сходства (логиты) в вероятностное распределение, где сумма весов внимания равна 1. Это гарантирует, что выходные значения остаются в разумном диапазоне, а модель фокусируется на наиболее релевантных элементах.  

**Масштабирование на $\sqrt{d_k}$** введено для контроля дисперсии. Если компоненты $Q_h$ и $K_h$ независимы и имеют дисперсию 1, то дисперсия их скалярного произведения $Q_h K_h^T$ равна $d_k$. Без масштабирования при больших $d_k$ аргументы softmax становятся экстремальными, градиенты насыщаются, и обучение замедляется.  

**Объединение голов**: выходы всех голов конкатенируются и проецируются обратно в пространство размерности $d_{\text{model}}$:  
$$  
\text{MultiHead}(X) = \text{Concat}(\text{head}_1, \ldots, \text{head}_H) W^O,  
$$  
где $W^O \in \mathbb{R}^{H d_v \times d_{\text{model}}}$ — обучаемая матрица. Размерности $d_k$ и $d_v$ обычно выбирают равными $d_{\text{model}} / H$, чтобы сохранить общую вычислительную сложность. Например, при $d_{\text{model}}=512$ и $H=8$, $d_k = d_v = 64$.  

**Почему именно такая размерность?**  
- Если бы $d_k$ и $d_v$ не уменьшались с ростом $H$, вычислительная сложность MultiHeadAttention росла бы квадратично: $O(H n^2 d_k)$. Сокращение $d_k$ и $d_v$ до $d_{\text{model}} / H$ сохраняет сложность на уровне $O(n^2 d_{\text{model}})$, как у одноголового внимания.  
- Проекция $W^O$ компенсирует уменьшение размерности в головах, возвращая выход в исходное пространство $d_{\text{model}}$, что необходимо для совместимости с другими слоями Transformer.  

**Почему именно эта формула?**  
1. **Разделение подпространств**: Каждая голова работает в своём $d_k$-мерном пространстве, что позволяет моделировать **независимые типы взаимодействий**. Например, одна голова может отслеживать согласование существительного с прилагательным, другая — ссылки на предыдущие предложения. Линейные проекции $W_h^Q, W_h^K, W_h^V$ "разделяют" исходные эмбеддинги на компоненты, которые легче интерпретировать в рамках конкретной задачи.  
2. **Параллелизм**: Независимость вычислений между головами позволяет эффективно распределять их на GPU.  
3. **Интерпретируемость**: Анализ весов внимания в разных головах (после обучения) позволяет выявить, какие типы паттернов выучила модель.  

**Пример вычисления для одной головы**:  
Пусть $X$ — матрица эмбеддингов размерности $n \times d_{\text{model}}$. Для головы $h$:  
- $Q_h = X W_h^Q$ ($n \times d_k$),  
- $K_h = X W_h^K$ ($n \times d_k$),  
- $V_h = X W_h^V$ ($n \times d_v$).  

Матрица весов внимания $A_h = \text{softmax}\left(\frac{Q_h K_h^T}{\sqrt{d_k}}\right)$ ($n \times n$) умножается на $V_h$, давая выход $A_h V_h$ ($n \times d_v$). Конкатенация выходов всех голов создаёт матрицу $n \times (H d_v)$, которая проецируется обратно в $n \times d_{\text{model}}$ через $W^O$.  

Критически важным стало **сохранение размерности**: выход MultiHeadAttention имеет ту же размерность $d_{\text{model}}$, что и вход, что позволяет стыковать его с остальными компонентами Transformer (нормализацией, feed-forward слоями) без дополнительных преобразований. Это также обеспечивает стабильность градиентов при глубоких архитектурах.  

Исторически, MultiHeadAttention стал ключевым отличием Transformer от предыдущих моделей с вниманием, таких как Pointer Networks. Его способность декомпозировать сложные зависимости в параллельные подзадачи сделала возможным обучение на больших корпусах текстов с сохранением интерпретируемости и вычислительной эффективности.  

In [4]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super().__init__()
        self.d_model = d_model
        self.num_heads = num_heads
        self.head_dim = d_model // num_heads
        
        assert self.head_dim * num_heads == d_model, "d_model must be divisible by num_heads"
        
        self.W_q = nn.Linear(d_model, d_model)
        self.W_k = nn.Linear(d_model, d_model)
        self.W_v = nn.Linear(d_model, d_model)
        self.W_o = nn.Linear(d_model, d_model)
        
    def scaled_dot_product_attention(self, Q, K, V, mask=None):
        # Q: [batch_size, num_heads, seq_len, head_dim]
        attn_scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.head_dim)
        
        if mask is not None:
            attn_scores = attn_scores.masked_fill(mask == 0, -1e9)
        
        attn_probs = F.softmax(attn_scores, dim=-1)
        output = torch.matmul(attn_probs, V)
        return output
        
    def forward(self, Q, K, V, mask=None):
        batch_size = Q.size(0)
        
        # Линейные преобразования
        Q = self.W_q(Q).view(batch_size, -1, self.num_heads, self.head_dim).transpose(1, 2)
        K = self.W_k(K).view(batch_size, -1, self.num_heads, self.head_dim).transpose(1, 2)
        V = self.W_v(V).view(batch_size, -1, self.num_heads, self.head_dim).transpose(1, 2)
        
        # Вычисление внимания
        attn_output = self.scaled_dot_product_attention(Q, K, V, mask)
        
        # Объединение голов
        attn_output = attn_output.transpose(1, 2).contiguous().view(batch_size, -1, self.d_model)
        
        # Финальное линейное преобразование
        output = self.W_o(attn_output)
        return output

## FeedForward 

Позиционно-зависимый Feed Forward слой в Transformer появился как ответ на необходимость **нелинейного преобразования признаков** после этапа внимания. MultiHeadAttention эффективно вычисляет глобальные зависимости между токенами, но для сложных задач (например, перевода) этого недостаточно: модель должна уметь комбинировать извлечённые паттерны и трансформировать их в новые семантические представления.  

Каждый токен последовательности независимо проходит через два линейных слоя. Первый слой расширяет пространство с $d_{\text{model}}$ (например, 512) до $d_{\text{ff}}$ (обычно 2048), применяя функцию активации ReLU:  
$$  
\text{hidden} = \text{ReLU}(x W_1 + b_1),  
$$  
где $W_1 \in \mathbb{R}^{d_{\text{model}} \times d_{\text{ff}}}$. 

Расширение размерности в 4 раза ($d_{\text{ff}} = 4d_{\text{model}}$) даёт модели достаточно параметров для обучения неочевидных комбинаций признаков. Второй слой возвращает представление в исходную размерность $d_{\text{model}}$:  
$$  
\text{output} = \text{hidden} W_2 + b_2,  
$$  
где $W_2 \in \mathbb{R}^{d_{\text{ff}} \times d_{\text{model}}}$. Между слоями добавляется dropout (например, с вероятностью 0.1) для регуляризации.  

**Почему именно так?**  
- **Нелинейность**: ReLU разрывает линейность, позволяя моделировать сложные функции. Без неё комбинация линейных слоёв сводилась бы к одному матричному умножению.  
- **Расширение-сжатие**: Увеличение размерности создаёт «бутылочное горлышко», вынуждая модель фильтровать шумовые признаки. Это похоже на работу autoencoder, но без потери информации, так как выход сохраняет исходную размерность.  
- **Позиционная независимость**: Обработка каждого токена отдельно компенсирует потенциальные потери локальных зависимостей после глобального внимания. Например, для фразы «синий шар» внимание может связать прилагательное и существительное, а FFN преобразует их совместное представление в вектор, кодирующий цвет и форму.  

Вход и выход слоя имеют одинаковую размерность $d_{\text{model}}$, что позволяет повторять блоки Encoder/Decoder. Dropout и остаточные соединения (реализованные вне этого слоя) стабилизируют обучение глубоких сетей. Исторически, эта архитектура заменила свёрточные слои из ранних моделей seq2seq, обеспечив более гибкое преобразование признаков без ограничений локальных рецептивных полей.  

In [5]:
class FeedForward(nn.Module):
    def __init__(self, d_model, d_ff=2048):
        super().__init__()
        self.linear1 = nn.Linear(d_model, d_ff)
        self.linear2 = nn.Linear(d_ff, d_model)
        self.dropout = nn.Dropout(0.1)
        
    def forward(self, x):
        x = self.dropout(F.relu(self.linear1(x)))
        x = self.linear2(x)
        return x

## EncoderLayer

До появления **EncoderLayer** в Transformer исследователи сталкивались с дилеммой: как совместить глобальное понимание контекста с локальными преобразованиями признаков, не теряя стабильности обучения в глубоких сетях. Ранние подходы, такие как RNN, страдали от исчезающих градиентов, а свёрточные сети требовали множества слоёв для захвата длинных зависимостей. Self-attention в Transformer решил проблему глобального контекста, но сам по себе не мог обеспечить сложную иерархическую обработку данных. Возник вопрос: как организовать последовательные преобразования, чтобы модель могла сначала выявить связи между токенами, а затем «переосмыслить» их, сохраняя устойчивость к глубине?  

**EncoderLayer** стал ответом — модулем, который объединяет два принципиальных этапа. Сначала входные эмбеддинги $x$, обогащённые позиционной информацией (Positional Encoding), проходят через **MultiHeadAttention**. Здесь каждый токен «спрашивает» остальные:  
$$  
\text{attn\_output} = \text{MultiHeadAttention}(x, x, x, mask),  
$$  
где $mask$ скрывает будущие токены (в декодере) или padding. Этот шаг позволяет, например, связать местоимение «он» с соответствующим существительным, даже если они разделены десятками слов. Но самовнимание — операция линейная в пространстве признаков. Чтобы добавить нелинейность и глубину, следом идёт **Feed Forward Network (FFN)** — два линейных слоя с расширением размерности:  
$$  
\text{ffn\_output} = \text{FFN}(x) = \text{ReLU}(x W_1 + b_1) W_2 + b_2.  
$$  
FFN действует как «мыслительный процесс»: преобразует глобальные зависимости, найденные вниманием, в новые семантические представления. Например, если внимание связало «яблоко» и «зелёное», FFN может закодировать это в вектор «фрукт + цвет».  

Но простое соединение этих шагов приводило к проблемам. Глубокие сети «забывали» исходные данные — градиенты исчезали, а признаки искажались. Здесь на помощь пришли **остаточные соединения** и **слойная нормализация**. После каждого подшага (самовнимание или FFN) к выходу добавляется исходный вход $x$, а результат нормализуется:  
$$  
x = \text{LayerNorm}(x + \text{Dropout}(\text{sublayer}(x))).  
$$  
Остатки работают как мосты, через которые градиенты и исходная информация свободно протекают даже через десятки слоёв. LayerNorm стабилизирует распределение активаций, вычисляя среднее и дисперсию по $d_{\text{model}}$-измерениям, что предотвращает «взрыв» или «затухание» значений.  

**Почему именно этот порядок?** Если бы FFN шёл до внимания, нелинейности ReLU могли бы «сломать» позиционную информацию, критичную для self-attention. А пост-нормализация (после остатка) вместо пре-нормализации (до подшага) выбрана не случайно: в оригинальном Transformer это заставляло градиенты проходить как через преобразованный, так и через исходный путь, балансируя обновления параметров.  

**Пример**: Эмбеддинг слова «bank» после самовнимания может получить признаки, связывающие его с «river» (банк как берег) или «money» (банк как учреждение). FFN преобразует эти связи в контекстно-зависимое представление, а остатки и нормализация сохраняют стабильность сигнала. Повторяясь через несколько EncoderLayer, модель итеративно уточняет смысл, как будто перечитывая текст, каждый раз замечая новые нюансы.  

Исторически, EncoderLayer стал шаблоном для масштабируемости. Его можно было повторять N раз (например, 6 или 12 слоёв), создавая глубокие модели без краха градиентов. Комбинация самовнимания и FFN оказалась настолько эффективной, что даже современные LLM, как GPT-4, сохраняют эту базовую структуру, дополняя её новыми механизмами.  

In [6]:
class EncoderLayer(nn.Module):
    def __init__(self, d_model, num_heads):
        super().__init__()
        self.self_attn = MultiHeadAttention(d_model, num_heads)
        self.ffn = FeedForward(d_model)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(0.1)
        
    def forward(self, x, mask=None):
        # Self attention
        attn_output = self.self_attn(x, x, x, mask)
        x = self.norm1(x + self.dropout(attn_output))
        
        # Feed forward
        ffn_output = self.ffn(x)
        x = self.norm2(x + self.dropout(ffn_output))
        return x

## DecoderLayer

Если EncoderLayer в Transformer научился понимать входной текст, собирая контекст в плотные векторы, то **DecoderLayer** возник из необходимости **генерировать выход** — слово за словом, учитывая и прошлые предсказания, и информацию от энкодера. Ранние подходы, как seq2seq с вниманием, уже связывали энкодер и декодер, но их рекуррентная природа ограничивала параллелизацию и способность захватывать сложные зависимости. В Transformer декодер должен был стать авторегрессионным, но без потери параллелизма — и здесь ключевым стал механизм **маскированного самовнимания** в сочетании с **кросс-вниманием**.  

**DecoderLayer** начинает работу с уже частично сгенерированной последовательности (например, переведённого предложения до текущего слова). Чтобы гарантировать, что модель не «подсматривает» будущие токены, применяется **маскированное самовнимание**:  
$$  
\text{attn\_output} = \text{MultiHeadAttention}(x, x, x, tgt\_mask),  
$$  
где $tgt\_mask$ — верхнетреугольная матрица с $-\\infty$ на позициях будущих токенов. При вычислении softmax это превращается в ноль, обнуляя их влияние. Например, при генерации третьего слова маска скрывает все токены после третьего, заставляя модель опираться только на уже созданный контекст.  

Но самовнимания недостаточно — декодер должен **соотносить выход с входом**. Для этого вводится **кросс-внимание**, где запросы ($Q$) берутся из декодера, а ключи ($K$) и значения ($V$) — из выхода энкодера:  
$$  
\text{cross\_attn\_output} = \text{MultiHeadAttention}(x, enc\_output, enc\_output, src\_mask).  
$$  
Здесь $src\_mask$ скрывает padding-токены исходной последовательности. Этот шаг работает как «опрос» энкодера: декодер «спрашивает», какие части входного текста релевантны текущему шагу генерации. Например, при переводе слова «apple» декодер через кросс-внимание связывает его с энкодеровскими «яблоко» или «компания», в зависимости от контекста.  

После кросс-внимания, как и в энкодере, следует **Feed Forward Network**, добавляющий нелинейность:  
$$  
\text{ffn\_output} = \text{FFN}(x).  
$$  
Каждый шаг сопровождается **остаточными соединениями** и **слойной нормализацией**:  
$$  
x = \text{LayerNorm}(x + \text{Dropout}(\text{sublayer}(x))),  
$$  
что сохраняет стабильность градиентов даже в глубоких сетях.  

**Почему именно три этапа?**  
1. **Маскированное самовнимание** изолирует уже сгенерированную часть последовательности, имитируя авторегрессию RNN.  
2. **Кросс-внимание** синхронизирует энкодер и декодер, позволяя последнему «заглядывать» в исходные данные — аналогично выравниванию в статистическом машинном переводе.  
3. **FFN** переосмысляет объединённую информацию, как финальный этап «принятия решения» о следующем токене.  

**Пример**: При переводе «I hit the bank» на русский, декодер:  
1. Через маскированное самовнимание связывает «я ударил» с «по», игнорируя будущие слова.  
2. Кросс-внимание находит в энкодере связь «bank» → «берег» (если контекст о реке) или «банк» (если о финансах).  
3. FFN преобразует это в «по берегу» или «в банк», сохраняя грамматику.  

**Исторически**, DecoderLayer стал мостом между пониманием (энкодер) и генерацией. Его способность параллельно обрабатывать последовательность, но авторегрессионно предсказывать токены, сделала Transformer универсальным каркасом для задач, требующих преобразования структур — от перевода до генерации кода. Каждый слой декодера — это шаг в «диалоге» между исходными данными и растущим выходом, где кросс-внимание выступает переводчиком, а FFN — редактором.  

In [7]:
class DecoderLayer(nn.Module):
    def __init__(self, d_model, num_heads):
        super().__init__()
        self.self_attn = MultiHeadAttention(d_model, num_heads)
        self.cross_attn = MultiHeadAttention(d_model, num_heads)
        self.ffn = FeedForward(d_model)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.norm3 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(0.1)
        
    def forward(self, x, enc_output, src_mask, tgt_mask):
        # Self attention (маскированное)
        attn_output = self.self_attn(x, x, x, tgt_mask)
        x = self.norm1(x + self.dropout(attn_output))
        
        # Cross attention (с выходом энкодера)
        attn_output = self.cross_attn(x, enc_output, enc_output, src_mask)
        x = self.norm2(x + self.dropout(attn_output))
        
        # Feed forward
        ffn_output = self.ffn(x)
        x = self.norm3(x + self.dropout(ffn_output))
        return x

## Transformer

Когда все компоненты Transformer — энкодер, декодер, механизмы внимания и позиционные кодировки — были разработаны, оставалась задача объединить их в единую модель, способную обучаться на парах последовательностей (например, исходный текст и перевод). Ранние подходы, такие как Seq2Seq, уже использовали разделение на энкодер и декодер, но их рекуррентная природа ограничивала параллелизацию и глубину. Архитектура Transformer, представленная в коде, стала итогом поиска баланса между выразительностью и вычислительной эффективностью.  

**Сборка модели** начинается с преобразования токенов в векторы. Эмбеддинги (`encoder_embedding` и `decoder_embedding`) отображают слова в пространство размерности $d_{\text{model}}$, а `positional_encoding` добавляет информацию о позициях:  
$$  
X_{\text{enc}} = \text{Embedding}(src) + \text{PositionalEncoding}(src),  
$$  
$$  
X_{\text{dec}} = \text{Embedding}(tgt) + \text{PositionalEncoding}(tgt).  
$$  
Без позиционного кодирования модель не смогла бы отличить перестановки слов, так как self-attention инвариантен к порядку.  

Далее энкодер и декодер собираются как **стек слоёв** (`num_layers`). Каждый слой в энкодере (`EncoderLayer`) последовательно уточняет представления входных данных: самовнимание находит глобальные зависимости, FFN добавляет нелинейность, а остатки и нормализация сохраняют устойчивость. Аналогично, декодер (`DecoderLayer`) поочерёдно применяет маскированное самовнимание, кросс-внимание к энкодеру и FFN. Многократное повторение этих слоёв позволяет модели итеративно улучшать представления, как бы «перечитывая» данные на разных уровнях абстракции.  

**Финальный слой** (`fc_out`) выполняет проекцию из $d_{\text{model}}$ в размер словаря целевого языка. Это преобразование интерпретирует векторы декодера как логиты — оценки вероятности каждого токена в словаре:  
$$  
\text{output} = W_{\text{out}} \cdot \text{dec\_output} + b_{\text{out}}.  
$$  
Softmax на выходе (не явно указанный в коде, но подразумеваемый в функции потерь) превращает логиты в распределение вероятностей, из которого выбирается следующее слово.  

**Почему именно так?**  
- **Глубина (`num_layers`)**: Каждый слой захватывает разные аспекты данных. Ранние слои энкодера могут выделять синтаксис, поздние — семантику. В декодере нижние слои отвечают за связь с энкодером, верхние — за грамматику вывода.  
- **Разделение эмбеддингов**: Разные матрицы для исходного и целевого языков позволяют модели работать с multilingual данными.  
- **Совместимость размерностей**: Все компоненты сохраняют размерность $d_{\text{model}}$, что упрощает обучение — градиенты свободно текут через остатки, а параметры обновляются согласованно. 

In [8]:
class Transformer(nn.Module):
    def __init__(self, src_vocab_size, tgt_vocab_size, d_model=512, num_heads=8, num_layers=6):
        super().__init__()
        self.encoder_embedding = nn.Embedding(src_vocab_size, d_model)
        self.decoder_embedding = nn.Embedding(tgt_vocab_size, d_model)
        self.positional_encoding = PositionalEncoding(d_model, dropout=0.1)
        
        self.encoder_layers = nn.ModuleList([EncoderLayer(d_model, num_heads) for _ in range(num_layers)])
        self.decoder_layers = nn.ModuleList([DecoderLayer(d_model, num_heads) for _ in range(num_layers)])
        
        self.fc_out = nn.Linear(d_model, tgt_vocab_size)
        
    def forward(self, src, tgt, src_mask=None, tgt_mask=None):
        # Энкодинг
        src_emb = self.positional_encoding(self.encoder_embedding(src))
        enc_output = src_emb
        for layer in self.encoder_layers:
            enc_output = layer(enc_output, src_mask)
        
        # Декодинг
        tgt_emb = self.positional_encoding(self.decoder_embedding(tgt))
        dec_output = tgt_emb
        for layer in self.decoder_layers:
            dec_output = layer(dec_output, enc_output, src_mask, tgt_mask)
        
        # Финальный слой
        output = self.fc_out(dec_output)
        return output

## Тест

In [9]:
def test_transformer():
    # Конфигурация
    torch.manual_seed(42)
    batch_size = 2
    seq_len = 10
    d_model = 512
    num_heads = 8
    src_vocab_size = 100
    tgt_vocab_size = 100
    num_layers = 2

    # Генерация синтетических данных
    src = torch.randint(0, src_vocab_size, (batch_size, seq_len))
    tgt = torch.randint(0, tgt_vocab_size, (batch_size, seq_len))
    
    # Генерация масок (пример)
    src_mask = torch.ones(batch_size, 1, 1, seq_len)  # Без маскирования
    tgt_mask = torch.tril(torch.ones(seq_len, seq_len)).expand(batch_size, 1, seq_len, seq_len)  # Каузальная маска

    # Инициализация модели
    transformer = Transformer(
        src_vocab_size=src_vocab_size,
        tgt_vocab_size=tgt_vocab_size,
        d_model=d_model,
        num_heads=num_heads,
        num_layers=num_layers
    )

    print("="*50)
    print("1. Тест Positional Encoding")
    pe = PositionalEncoding(d_model, dropout=0.1)
    x = torch.randn(1, seq_len, d_model)
    print(f"До PE: mean={x.mean().item():.4f}, std={x.std().item():.4f}")
    x_pe = pe(x)
    print(f"После PE: mean={x_pe.mean().item():.4f}, std={x_pe.std().item():.4f}")
    print(f"Форма PE: {x_pe.shape} (должна быть [1, {seq_len}, {d_model}])")
    
    print("\n2. Тест Multi-Head Attention")
    mha = MultiHeadAttention(d_model, num_heads)
    q = k = v = torch.randn(batch_size, seq_len, d_model)
    attn_output = mha(q, k, v)
    print(f"Форма выхода внимания: {attn_output.shape} (должна быть {q.shape})")
    print(f"Максимальное значение: {attn_output.max().item():.4f}")
    print(f"Минимальное значение: {attn_output.min().item():.4f}")

    print("\n3. Тест Encoder Layer")
    encoder_layer = EncoderLayer(d_model, num_heads)
    enc_input = torch.randn(batch_size, seq_len, d_model)
    enc_output = encoder_layer(enc_input)
    print(f"Форма выхода энкодера: {enc_output.shape} (должна быть {enc_input.shape})")
    print(f"Изменение данных: {torch.allclose(enc_input, enc_output, atol=1e-4)} (должно быть False)")

    print("\n4. Тест Decoder Layer")
    decoder_layer = DecoderLayer(d_model, num_heads)
    dec_input = torch.randn(batch_size, seq_len, d_model)
    dec_output = decoder_layer(dec_input, enc_output, src_mask, tgt_mask)
    print(f"Форма выхода декодера: {dec_output.shape} (должна быть {dec_input.shape})")
    print(f"Норма выходных данных: {dec_output.norm().item():.4f}")

    print("\n5. Полный тест Transformer")
    print("Входные данные:")
    print(f"src: {src.shape} (max={src.max().item()}, min={src.min().item()})")
    print(f"tgt: {tgt.shape} (max={tgt.max().item()}, min={tgt.min().item()})")
    
    output = transformer(src, tgt, src_mask, tgt_mask)
    print("\nПроверка формы выхода:")
    print(f"Ожидаемая форма: ({batch_size}, {seq_len}, {tgt_vocab_size})")
    print(f"Реальная форма:   {output.shape}")
    
    print("\nПроверка градиентов:")
    dummy_loss = output.sum()
    dummy_loss.backward()
    has_gradients = any(p.grad is not None for p in transformer.parameters())
    print(f"Градиенты вычислены: {has_gradients} (должно быть True)")

    print("\n6. Проверка параметров модели:")
    total_params = sum(p.numel() for p in transformer.parameters())
    print(f"Всего параметров: {total_params}")
    print(f"Параметры энкодера: {sum(p.numel() for p in transformer.encoder_embedding.parameters())}")
    print(f"Параметры декодера: {sum(p.numel() for p in transformer.decoder_embedding.parameters())}")

    print("\nТест завершен!")

In [10]:
test_transformer()

1. Тест Positional Encoding
До PE: mean=-0.0316, std=1.0065
После PE: mean=0.4559, std=1.2109
Форма PE: torch.Size([1, 10, 512]) (должна быть [1, 10, 512])

2. Тест Multi-Head Attention
Форма выхода внимания: torch.Size([2, 10, 512]) (должна быть torch.Size([2, 10, 512]))
Максимальное значение: 0.4148
Минимальное значение: -0.4277

3. Тест Encoder Layer
Форма выхода энкодера: torch.Size([2, 10, 512]) (должна быть torch.Size([2, 10, 512]))
Изменение данных: False (должно быть False)

4. Тест Decoder Layer
Форма выхода декодера: torch.Size([2, 10, 512]) (должна быть torch.Size([2, 10, 512]))
Норма выходных данных: 101.1924

5. Полный тест Transformer
Входные данные:
src: torch.Size([2, 10]) (max=95, min=6)
tgt: torch.Size([2, 10]) (max=99, min=10)

Проверка формы выхода:
Ожидаемая форма: (2, 10, 100)
Реальная форма:   torch.Size([2, 10, 100])

Проверка градиентов:
Градиенты вычислены: True (должно быть True)

6. Проверка параметров модели:
Всего параметров: 14866532
Параметры энкодера: 5